In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import pandas_ta as ta

dataF = yf.download("EURUSD=X", start="2022-11-13", end="2023-01-11", interval='1h')
dataF.iloc[:,:]

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
2022-11-14 00:00:00,1.033058,1.033805,1.032738,1.033485,1.033485,0
2022-11-14 01:00:00,1.033592,1.035090,1.032524,1.032951,1.032951,0
2022-11-14 02:00:00,1.033058,1.033271,1.032098,1.033058,1.033058,0
2022-11-14 03:00:00,1.032951,1.033699,1.032631,1.032844,1.032844,0
2022-11-14 04:00:00,1.032844,1.032844,1.030928,1.031566,1.031566,0
...,...,...,...,...,...,...
2023-01-10 19:00:00,1.074114,1.074807,1.073998,1.074345,1.074345,0
2023-01-10 20:00:00,1.074229,1.074807,1.073307,1.073883,1.073883,0
2023-01-10 21:00:00,1.073883,1.074229,1.073653,1.073998,1.073998,0
2023-01-10 22:00:00,1.073768,1.074576,1.073537,1.073883,1.073883,0


In [2]:
#!pip install bokeh==2.4.3

In [3]:
grid_distance = 0.005
TPSL_Ratio = 1
midprice = 1.065

In [4]:
def generate_grid(midprice, grid_distance, grid_range):
    return (np.arange(midprice-grid_range, midprice+grid_range, grid_distance))

grid = generate_grid(midprice=midprice, grid_distance=grid_distance, grid_range=0.1)
grid


array([0.965, 0.97 , 0.975, 0.98 , 0.985, 0.99 , 0.995, 1.   , 1.005,
       1.01 , 1.015, 1.02 , 1.025, 1.03 , 1.035, 1.04 , 1.045, 1.05 ,
       1.055, 1.06 , 1.065, 1.07 , 1.075, 1.08 , 1.085, 1.09 , 1.095,
       1.1  , 1.105, 1.11 , 1.115, 1.12 , 1.125, 1.13 , 1.135, 1.14 ,
       1.145, 1.15 , 1.155, 1.16 , 1.165])

In [5]:
signal = [0]*len(dataF)
i=0
for index, row in dataF.iterrows():
    for p in grid:
        if min(row.Low, row.High)<p and max(row.Low, row.High)>p:
            signal[i]=1
    i+=1

In [6]:
dataF["signal"]=signal
dataF[dataF["signal"]==1]

,Open,High,Low,Close,Adj Close,Volume,signal
2022-11-14 01:00:00,1.033592,1.035090,1.032524,1.032951,1.032951,0,1
2022-11-14 08:00:00,1.033378,1.035197,1.032844,1.034447,1.034447,0,1
2022-11-14 10:00:00,1.031566,1.032098,1.029124,1.029654,1.029654,0,1
2022-11-14 12:00:00,1.028595,1.033699,1.027538,1.033699,1.033699,0,1
2022-11-14 16:00:00,1.033271,1.035304,1.032844,1.033699,1.033699,0,1
...,...,...,...,...,...,...,...
2023-01-09 19:00:00,1.074576,1.075732,1.074576,1.075732,1.075732,0,1
2023-01-09 20:00:00,1.075269,1.075616,1.073422,1.073422,1.073422,0,1
2023-01-10 09:00:00,1.074114,1.075384,1.073307,1.074807,1.074807,0,1
2023-01-10 14:00:00,1.072616,1.076079,1.072616,1.074345,1.074345,0,1


In [7]:
dfpl = dataF[:].copy()
def SIGNAL():
    return dfpl.signal
dfpl['ATR'] = ta.atr(high = dfpl.High, low = dfpl.Low, close = dfpl.Close, length = 16)
dfpl.dropna(inplace=True)

In [8]:
from backtesting import Strategy
from backtesting import Backtest
import backtesting

class MyStrat(Strategy):
    mysize = 0.5
    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = self.data.ATR[-1] #grid_distance
        TPSLRatio = 1.2*TPSL_Ratio

        if self.signal1==1 and len(self.trades)<=2:   
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)
            
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(dfpl, MyStrat, cash=100, margin=1/1, commission=.000)
stat = bt.run()
stat

C:\Users\INNOVACION\anaconda3\envs\AI\lib\site-packages\backtesting\_plotting.py:50: UserWarning: Jupyter Notebook detected. Setting Bokeh output to notebook. This may not work in Jupyter clients without JavaScript support (e.g. PyCharm, Spyder IDE). Reset with `backtesting.set_bokeh_output(notebook=False)`.
  warnings.warn('Jupyter Notebook detected. '


Loading BokehJS ...

Start                     2022-11-14 16:00:00
End                       2023-01-10 23:00:00
Duration                     57 days 07:00:00
Exposure Time [%]                   65.853659
Equity Final [$]                    102.10384
Equity Peak [$]                     102.25334
Return [%]                            2.10384
Buy & Hold Return [%]                3.909775
Return (Ann.) [%]                   14.182781
Volatility (Ann.) [%]                2.855075
Sharpe Ratio                         4.967568
Sortino Ratio                       11.625148
Calmar Ratio                        19.040081
Max. Drawdown [%]                   -0.744891
Avg. Drawdown [%]                   -0.113016
Max. Drawdown Duration       14 days 09:00:00
Avg. Drawdown Duration        1 days 06:00:00
# Trades                                  584
Win Rate [%]                        35.273973
Best Trade [%]                       0.434259
Worst Trade [%]                     -0.358497
Avg. Trade [%]                    

In [9]:
backtesting.set_bokeh_output(notebook=False)
bt.plot(show_legend=False, plot_width=None, plot_equity=True, plot_return=False, 
plot_pl=False, plot_volume=False, plot_drawdown=False, smooth_equity=False, relative_equity=True, 
superimpose=True, resample=False, reverse_indicators=False, open_browser=True)

Row(id='1352', ...)

In [10]:
stat._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,24,2,2,1.033805,1.033805,0.000000,0.000000,2022-11-14 18:00:00,2022-11-14 18:00:00,0 days 00:00:00
1,18,2,3,1.033805,1.035518,0.030830,0.001657,2022-11-14 18:00:00,2022-11-14 19:00:00,0 days 01:00:00
2,6,2,4,1.033805,1.035625,0.010920,0.001760,2022-11-14 18:00:00,2022-11-14 20:00:00,0 days 02:00:00
3,7,3,4,1.035518,1.035625,0.000750,0.000103,2022-11-14 19:00:00,2022-11-14 20:00:00,0 days 01:00:00
4,27,4,5,1.035625,1.033118,-0.067696,-0.002421,2022-11-14 20:00:00,2022-11-14 21:00:00,0 days 01:00:00
...,...,...,...,...,...,...,...,...,...,...
579,23,957,974,1.073653,1.075727,0.047711,0.001932,2023-01-09 21:00:00,2023-01-10 14:00:00,0 days 17:00:00
580,24,975,975,1.074460,1.074460,0.000000,0.000000,2023-01-10 15:00:00,2023-01-10 15:00:00,0 days 00:00:00
581,23,975,975,1.074460,1.072635,-0.041968,-0.001698,2023-01-10 15:00:00,2023-01-10 15:00:00,0 days 00:00:00
582,24,976,976,1.073076,1.073076,0.000000,0.000000,2023-01-10 16:00:00,2023-01-10 16:00:00,0 days 00:00:00
